Integrate with Data & Analytcs services in AWS¶
====================

As Data Lakes are becoming more popular and democratizing data is becoming the new normal we want to be able to get the raw data from the Data Lake into the hands of Data Scientists.

To do this we need the ability to integrate with common scm tools as well as discover data in the data lake. To do this we will be using AWS Glue and Athena to discover the data that can then be used in training models in SageMaker.

Set up the environment get acces to the Python boto3 library and get a reference to the AWS Glue Service.

In [ ]:
import boto3

client = boto3.client('glue')

With boto3 we can use the get_tables call to get a json response of the tables in the taxidb in the AWS Glue Data Catalog.

In [ ]:
response = client.get_tables(
    DatabaseName='<<GlueDatabaseName>>',
)

response

To get a better view of just the tables and their location in S3 we can get just the table list and location.

In [ ]:
print("table_name\tlocation")
for table in response.get('TableList'):
    print(table.get('Name') + '\t\t' + table.get('StorageDescriptor').get('Location') )

Finally we can get the table metadata including the columns in the AWS Glue Data Catalog for the 'yellow' table in the 'taxidb'.

In [ ]:
response = client.get_table(
    DatabaseName='<<GlueDatabaseName>>',
    Name='yellow'
)

response

We installed the pyathena library in our custom environment and can now query the AWS Glue Data Catalog using Athena.

In [ ]:
from pyathena import connect
from pyathena.util import as_pandas

cursor = connect(region_name='<<Region>>', s3_staging_dir='<<Staging Dir>>').cursor()

cursor.execute('select pickup_latitude, pickup_longitude, total_amount, trip_distance from taxidb.yellow limit 10')

df = as_pandas(cursor)
df.head(10)